In [ ]:
!pip install gradio
!pip install sentence-transformers
!pip install rapidfuzz

Import Libraies

In [2]:
import gradio as gr
import torch
import torchvision.transforms.functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageOps
import cv2
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import torch.nn as nn
import os


In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
model_crnn_path = "/content/drive/MyDrive/Individual_Research_Project/models/crnn_prescription_final.pth"
model_frcnn_path = "/content/drive/MyDrive/Individual_Research_Project/models/faster_rcnn_words_new_1.pth"
kb_main_path = "/content/drive/MyDrive/Individual_Research_Project/knowledge_base.xlsx"
kb_side_path = "/content/drive/MyDrive/Individual_Research_Project/SideEffects.csv"
kb_shortform_path = "/content/drive/MyDrive/Individual_Research_Project/short_form_mapping.csv"


Label Encoder and CRNN Model

In [4]:
class LabelEncoder:
    def __init__(self):
        self.characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.,:/()"
        self.char2idx = {char: idx + 1 for idx, char in enumerate(self.characters)}
        self.idx2char = {idx + 1: char for idx, char in enumerate(self.characters)}
        self.blank_label = 0

    def encode(self, text):
        return [self.char2idx[char] for char in text if char in self.char2idx]

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices if idx != self.blank_label])

    def get_vocab_size(self):
        return len(self.char2idx) + 1

class CRNN(nn.Module):
    def __init__(self, img_height, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.ReLU()
        )
        self.rnn1 = nn.LSTM(512, 256, batch_first=True, bidirectional=True)
        self.rnn2 = nn.LSTM(512, 256, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        assert h == 1
        x = x.squeeze(2)
        x = x.permute(0, 2, 1)
        x, _ = self.rnn1(x)
        x, _ = self.rnn2(x)
        return self.fc(x)

Initialize Models and Knowledge Bases

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models
model_detector = fasterrcnn_resnet50_fpn(weights=None, num_classes=2)
model_detector.load_state_dict(torch.load(model_frcnn_path, map_location=device))
model_detector = model_detector.to(device).eval()

encoder = LabelEncoder()
model_crnn = CRNN(img_height=32, num_classes=encoder.get_vocab_size())
model_crnn.load_state_dict(torch.load(model_crnn_path, map_location=device))
model_crnn = model_crnn.to(device).eval()

# Load knowledge bases
kb_main_df = pd.read_excel(kb_main_path).dropna(subset=["Entry", "Category"])
kb_main_entries = kb_main_df["Entry"].astype(str).str.lower().tolist()
kb_main_categories = kb_main_df["Category"].str.lower().tolist()

kb_side_df = pd.read_csv(kb_side_path)
kb_side_df.columns = kb_side_df.columns.str.strip().str.lower().str.replace(" ", "_")
kb_side_df["drug_name"] = kb_side_df["drug_name"].str.strip().str.lower()
kb_side_df["common_usage"] = kb_side_df["common_usage"].fillna("Not Available")
kb_side_df["side_effects"] = kb_side_df["side_effects"].fillna("Not Available")

short_form_df = pd.read_csv(kb_shortform_path)
short_form_df.columns = short_form_df.columns.str.strip().str.lower().str.replace(" ", "_")
short_to_full = dict(zip(
    short_form_df["abbreviation"].str.lower(),
    short_form_df["full_drug_name"]
))

# Load sentence transformer
model_st = SentenceTransformer("all-MiniLM-L6-v2")
kb_main_embeddings = model_st.encode(kb_main_entries, convert_to_tensor=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Helper Functions

In [6]:
def resize_and_pad(img, size=(128, 32)):
    img = img.convert("L")
    w, h = img.size
    scale = size[1] / h
    new_w = int(w * scale)
    img = img.resize((new_w, size[1]), Image.BILINEAR)
    if new_w > size[0]:
        img = img.resize(size, Image.BILINEAR)
    else:
        pad = (size[0] - new_w) // 2
        img = ImageOps.expand(img, (pad, 0, size[0] - new_w - pad, 0), fill=255)
    return img

def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY, 11, 2)
    blurred = cv2.GaussianBlur(thresh, (1, 1), 0)
    return cv2.cvtColor(blurred, cv2.COLOR_GRAY2RGB)

def predict_word(crop):
    crop = preprocess_image(crop)
    crop = resize_and_pad(Image.fromarray(crop))
    image = F.to_tensor(crop).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model_crnn(image)
        output = output.softmax(2)
        pred_idx = torch.argmax(output, dim=2).squeeze().tolist()
        return encoder.decode(pred_idx).strip()

def expand_short_form(word):
    return short_to_full.get(word.lower(), word)

def correct_spell(word, kb_entries, threshold=80):
    best, score, _ = process.extractOne(word, kb_entries, scorer=fuzz.ratio)
    return best if score >= threshold else word

def match_to_kb(pred_text, kb_entries, kb_embeddings, threshold=0.6):
    if not pred_text or not isinstance(pred_text, str):
        return "No Match", "N/A", 0.0

    pred_text = pred_text.lower().strip()
    emb = model_st.encode(pred_text, convert_to_tensor=True)
    sim = util.cos_sim(emb, kb_embeddings)[0]
    best_idx = int(sim.argmax())

    if float(sim[best_idx]) >= threshold:
        matched_entry = kb_main_df.iloc[best_idx]["Entry"]  # Original case
        return matched_entry, kb_main_categories[best_idx], float(sim[best_idx])

    # Fallback to fuzzy matching
    best_match, score, _ = process.extractOne(
        pred_text,
        kb_entries,
        scorer=fuzz.token_set_ratio
    )
    if score >= 75:
        matched_idx = kb_entries.index(best_match)
        matched_entry = kb_main_df.iloc[matched_idx]["Entry"]
        return matched_entry, kb_main_categories[matched_idx], float(score/100)

    return "No Match", "N/A", float(sim[best_idx])

def get_side_info(drug_name):
    if not isinstance(drug_name, str) or not drug_name.strip():
        return "Not Available", "Not Available"

    drug_name = drug_name.lower().strip()

    # Exact match
    exact_match = kb_side_df[kb_side_df["drug_name"] == drug_name]
    if not exact_match.empty:
        return (
            exact_match["common_usage"].values[0],
            exact_match["side_effects"].values[0]
        )

    # Partial match
    partial_matches = kb_side_df[
        kb_side_df["drug_name"].str.contains(drug_name)
    ]
    if not partial_matches.empty:
        best_match = process.extractOne(
            drug_name,
            partial_matches["drug_name"],
            scorer=fuzz.ratio
        )
        if best_match and best_match[1] > 70:
            matched_row = partial_matches[
                partial_matches["drug_name"] == best_match[0]]
            return (
                matched_row["common_usage"].values[0],
                matched_row["side_effects"].values[0]
            )

    return "Not Available", "Not Available"

Main Processing Function

In [10]:
def run_prescription_ocr(image):
    if image is None:
        return pd.DataFrame([{"Error": "Please upload a prescription image."}])

    try:
        img = image.convert("RGB")
        img_tensor = F.to_tensor(img).unsqueeze(0).to(device)

        with torch.no_grad():
            prediction = model_detector(img_tensor)

        boxes = prediction[0]["boxes"].detach().cpu().numpy()
        scores = prediction[0]["scores"].detach().cpu().numpy()
        selected_boxes = boxes[scores > 0.4]
        img_cv = np.array(img)
        img_h, img_w, _ = img_cv.shape

        results = []
        for box in selected_boxes:
            x1, y1, x2, y2 = box.astype(int)
            pad = 10
            x1, y1 = max(0, x1 - pad), max(0, y1 - pad)
            x2, y2 = min(img_w, x2 + pad), min(img_h, y2 + pad)
            crop = img_cv[y1:y2, x1:x2]

            # Processing pipeline
            raw_text = predict_word(crop)
            expanded = expand_short_form(raw_text)
            corrected = correct_spell(expanded.lower(), kb_main_entries)
            matched_entry, category, sim = match_to_kb(corrected, kb_main_entries, kb_main_embeddings)
            usage, side_effects = get_side_info(matched_entry if matched_entry != "No Match" else corrected)

            results.append({
                "Raw Text": raw_text,
                "Expanded Form": expanded,
                "Corrected Text": corrected,
                "Matched Drug": matched_entry,
                "Category": category,
                "Confidence": f"{sim:.2f}",
                "Common Usage": usage,
                "Side Effects": side_effects
            })

        if not results:
            return pd.DataFrame([{"Note": "No words detected. Try another image."}])

        return pd.DataFrame(results).sort_values("Confidence", ascending=False)

    except Exception as e:
        return pd.DataFrame([{"Error": f"Processing error: {str(e)}"}])

Gradio Interface

In [ ]:
css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.output-dataframe {
    font-size: 14px;
    max-height: 500px;
    overflow-y: auto;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("Prescription Drug Recognition System")
    gr.Markdown("Upload a handwritten prescription to identify medications and view their details.")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Prescription Image")
            submit_btn = gr.Button("Analyze", variant="primary")
        with gr.Column():
            output_table = gr.Dataframe(
                headers=["Raw Text", "Expanded Form", "Corrected Text",
                        "Matched Drug", "Category", "Confidence",
                        "Common Usage", "Side Effects"],
                datatype=["str", "str", "str", "str", "str", "str", "str", "str"],
                col_count=(8, "fixed"),
                wrap=True,
                interactive=False
            )

    gr.Examples(
        examples=[],
        inputs=image_input,
        outputs=output_table,
        fn=run_prescription_ocr
    )

    submit_btn.click(
        fn=run_prescription_ocr,
        inputs=image_input,
        outputs=output_table
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
